# DomoJupyter (GetInstanceCredentials)

> a function for interacting with DomoJupyter Accounts and Credentials


In [1]:
# | default_exp integrations.DomoJupyter


In [2]:
# | exporti
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum


import re
import time
import json

import pandas as pd

from fastcore.basics import patch_to

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.Logger as lc
import domolibrary.classes.DomoDataset as dmds
import domolibrary.utils.chunk_execution as ce


# get_jupyter_account

basic function that uses the wraps the domojupyter get_account_property functions in a loop.
Addresses an issue where sometimes DomoJupyter throws an error


In [3]:
# | export
class GetJupyter_ErrorRetrievingAccount(Exception):
    def __init__(self, account_name):
        self.message = f"failure to retrieve DomoDomoJupyter Account {account_name}"
        super().__init__(self.message)


class GetJupyter_ErrorRetrievingAccountProperty(Exception):
    def __init__(self, account_name, property_name):
        self.message = f"failure to retrieve {property_name} DomoDomoJupyter Account {account_name}"
        super().__init__(self.message)


def get_jupyter_account(
    account_name: str,  # name of account as it appears in the
    domojupyter_fn: callable,
    maximum_retry: int = 10,
) -> (
    list,
    dict,
):  # returns account properties list and a dictionary of the properties.
    """import a domojupyter account, will loop until success"""
    account_properties = None

    retry_attempt = 0
    while not account_properties and retry_attempt <= maximum_retry:
        try:
            account_properties = domojupyter_fn.get_account_property_keys(account_name)
            retry_attempt += 1

        except Exception as e:
            print(f"Error:  retry attempt {retry_attempt} - {account_name}: {e}")
            time.sleep(2)

    if not account_properties:
        raise GetJupyter_ErrorRetrievingAccount(account_name=account_name)

    obj = {}

    retry_attempt = 0
    for index, prop in enumerate(account_properties):
        value = None

        while not value and retry_attempt <= maximum_retry:
            try:
                value = domojupyter_fn.get_account_property_value(
                    account_name, account_properties[index]
                )

            except Exception as e:
                print(f"trying again - {prop} - {e}")
                time.sleep(2)

        if not value:
            raise GetJupyter_ErrorRetrievingAccountProperty(
                account_name=account_name, property_name=prop
            )

        obj.update({prop: value})

    return account_properties, obj

In [4]:
# | export


class NoConfigCompanyError(Exception):
    def __init__(self, sql, domo_instance):
        message = f'SQL "{sql}" returned no results in {domo_instance}'
        self.message = message
        super().__init__(self.message)


class GetInstanceConfig:
    config: pd.DataFrame = None
    logger: lc.Logger = None

    def __init__(self, logger: Optional[lc.Logger] = None):
        self.logger = logger or lc.Logger(app_name="GetInstanceConfig")

    async def _retrieve_company_ds(
        self,
        config_auth: dmda.DomoAuth,
        dataset_id: str,
        sql: str,
        debug_prn: bool = False,
        debug_api: bool = False,
        debug_log: bool = False,
        debug_num_stacks_to_drop: int = 2,
    ) -> [dict]:  # list of config query
        """wrapper for `DomoDataset.query_dataset_private` retrieves company configuration dataset and stores it as config"""

        ds = await dmds.DomoDataset.get_from_id(
            auth=config_auth,
            dataset_id=dataset_id,
            debug_api=debug_api,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=self.__class__.__name__,
        )

        message = (
            f"⚙️ START - Retrieving company list \n{ds.display_url()} using \n{sql}"
        )

        if debug_prn:
            print(message)

        self.logger.log_info(message, debug_log=debug_log)

        config_df = await ds.query_dataset_private(
            auth=config_auth,
            dataset_id=dataset_id,
            sql=sql,
            debug_api=debug_api,
            loop_until_end=True,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=self.__class__.__name__,
            is_return_dataframe = False
        )
        if len(config_df) == 0:
            raise NoConfigCompanyError(sql, domo_instance=config_auth.domo_instance)

        self.config = config_df

        message = f"\n⚙️ SUCCESS 🎉 Retrieved company list \nThere are {len(config_df)} companies to update"

        if debug_prn:
            print(message)
        self.logger.log_info(message, debug_log=debug_log)

        return config_df

In [5]:
# | hide
# sample implementation of _retrieve_company_data
import os

config_auth = dmda.DomoTokenAuth(
    domo_instance="domo-DomoJupyter",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

logger = lc.Logger(app_name="test_retrieve_company")

gic = GetInstanceConfig(logger=logger)

await gic._retrieve_company_ds(
    config_auth=config_auth,
    dataset_id="8d2a8055-7918-4039-b67d-361647e96ea8",
    sql="SELECT domain from Table",
    debug_prn=True,
    debug_log=False,
    debug_api=False,
)

pd.DataFrame(gic.config)
# pd.DataFrame(logger.logs)

⚙️ START - Retrieving company list 
https://domo-DomoJupyter.domo.com/datasources/8d2a8055-7918-4039-b67d-361647e96ea8/details/overview using 
SELECT domain from Table
adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 18, 'module_index': 12, 'num_stacks_to_drop_passed': 5}

⚙️ SUCCESS 🎉 Retrieved company list 
There are 1 companies to update
adjusting num_stacks_to_drop, consider revising `get_traceback` call
{'stack_length': 18, 'module_index': 12, 'num_stacks_to_drop_passed': 5}


,domain
0,domo-dojo


# Get Domo Jupyter Account objects

A Class for converting DomoJupyter Account objects into DomoAuth account objects


In [6]:
# | export
class InvalidAccountTypeError(Exception):
    """raised when account type is not expected type"""

    def __init__(self, account_name, account_type):

        self.message = f"account: {account_name} is not {account_type}"
        super().__init__(self.message)


@dataclass
class DomoJupyterAccount_InstanceAuth:
    """class for interacting with DomoJupyterAccount objects and generating a DomoAuth object"""

    account_name: str

    domo_username: str = None

    display_name: str = field(repr=False, default=None)

    domo_instance: str = field(repr=False, default=None)
    domo_instance_ls: list = field(repr=False, default=None)

    raw_cred: dict = field(repr=False, default=None)
    domo_password: str = field(repr=False, default=None)
    domo_access_token: str = field(repr=False, default=None)

    auth_ls: list = field(repr=False, default=None)

    account_name_mask = "^dj_.*_acc"

    def __post_init__(self):
        if not self.display_name and self.domo_username:
            self._set_display_name()

    def _set_display_name(self):
        clean_text = re.sub("@.*$", "", self.domo_username)
        self.display_name = clean_text

    @staticmethod
    def _test_regex_mask(
        test_string: str,  # the string to test
        regex_mask: str,  # the regex expression to test
    ) -> bool:  # boolean of the re match
        """tests if a string matches the regex pattern"""

        return bool(re.match(regex_mask, test_string))

    @staticmethod
    def _clean_account_admin_accounts(account_name):

        clean_str = re.sub("^dj_", "", account_name)
        clean_str = re.sub("_acc$", "", clean_str)

        return clean_str

# Get Domains with Instance Config

Use this method to configure a dataset that retrieves a list of domains from a config instance (using config credentials). Pass an `auth_enum` object to enumerate different authenticaiton variations to expect in the result dataset (see example).

Theoretically, each of the enumerated auth variations should already exist in the instance.

The Config Dataset must return columns **domo_instance** and **auth_match_col**


In [7]:
# | export
async def is_v2(instance_auth: dmda.DomoFullAuth):
    """wrapper for the domo boostrap.is_group_ownership_beta function to return a binary for if the instance has the group ownership beta enabled"""

    import domolibrary.classes.DomoBootstrap as dmbs

    domo_bootstrap = dmbs.DomoBootstrap(auth=instance_auth)
    is_v2 = await domo_bootstrap.is_group_ownership_beta()

    return 1 if is_v2 else 0


async def process_row(
    instance: dict,
    domo_instance,
    instance_creds: DomoJupyterAccount_InstanceAuth,
    config_enum=None,
    debug_api: bool = False,
    debug_prn: bool = False,
    logger = None
):
    # convert DomoJupyterAccount_InstanceAuth obj to
    if isinstance(instance_creds, DomoJupyterAccount_InstanceAuth):
        instance_creds = instance_creds.generate_auth(domo_instance=domo_instance)

    if isinstance(instance_creds, dmda.DomoAuth):
        instance.update({"instance_auth": instance_creds})

    try:
        await instance_creds.get_auth_token(debug_api=debug_api)
        instance.update(
            {"is_valid": 1}
        )

        if isinstance(instance_creds, dmda.DomoFullAuth):
            instance.update({"is_v2": await is_v2(instance_auth=instance_creds)})

    except (dmda.InvalidCredentialsError, dmda.AccountLockedError) as e:
        if debug_prn:
            print(e)
        logger.log_error(str(e))
        instance.update({"is_valid": 0})

    config_creds = None
    if config_enum and instance.get("config_exception_pw") == 0:
        config_creds = config_enum["config_1"].value

    elif config_enum and instance.get("config_exception_pw") == 1:
        config_creds = config_enum["config_0"].value

    if isinstance(config_creds, DomoJupyterAccount_InstanceAuth):
        config_creds = config_creds.generate_auth(domo_instance=domo_instance)

    if isinstance(config_creds, dmda.DomoAuth):
        instance.update({"config_auth": config_creds})

#### sample implementaiton of is_v2

In [8]:
import os
import domolibrary.client.DomoAuth as dmda

config_auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

await config_auth.print_is_token()

await is_v2(instance_auth=config_auth)




🎉 full_auth token retrieved from domo-community ⚙️


0

In [9]:
# | export


class GetDomains_Query_AuthMatch_Error(Exception):
    """raise if SQL query fails to return column named 'auth_match_col'"""

    def __init__(self, sql: str = None, domo_instance: str = None, message: str = None):
        self.message = (
            message
            or f"Query failed to return a column 'auth_match_col' sql = {sql} in {domo_instance}"
        )
        super().__init__(self.message)


@patch_to(GetInstanceConfig, cls_method=True)
async def get_domains_with_instance_auth(
    cls: GetInstanceConfig,
    default_auth: dmda.DomoAuth,  # default auth to use with each row
    auth_enum: Enum,  # Enum where enum_name should match to `auth_match_col` from config_sql query and enum_value is the appropriate DomoAuth or DomoJupyterAccount object
    config_auth: dmda.DomoAuth = None,  # which instance to retrieve configuration data from
    config_dataset_id: str = None,  # dataset_id to run config_sql query against
    config_sql: str = "select domain as domo_instance,concat(config_useprod, '-', project) as auth_match_col from table",
    debug_api: bool = False,
    debug_log: bool = False,
    debug_prn: bool = False,
    logger: lc.Logger = None,  # pass in Logger class
) -> pd.DataFrame:  # returns a dataframe with domo_instance, instance_auth, and binary column is_valid
    """uses a sql query to retrieve a list of domo_instances and map authentication object to each instance"""

    if not logger:
        logger = lc.Logger(app_name="get_domains_with_instance_auth")

    gic = cls(logger=logger)

    config_ls = await gic._retrieve_company_ds(
        config_auth=config_auth,
        dataset_id=config_dataset_id,
        sql=config_sql,
        debug_prn=debug_prn,
        debug_log=debug_log,
        debug_api=debug_api,
    )

    if "auth_match_col" not in config_ls[0]:
        message = f"Query failed to return a column 'auth_match_col' sql = {config_sql} in {config_auth.domo_instance}"
        raise GetDomains_Query_AuthMatch_Error(message)

    await ce.gather_with_concurrency(*[
        process_row(
            instance=instance,
            domo_instance = instance['domo_instance'],
            instance_creds=auth_enum[instance['auth_match_col']].value
            if instance['auth_match_col'] in auth_enum._member_names_
            else default_auth,
            config_enum=auth_enum if "config_1" in auth_enum._member_names_ else None,
            debug_api = debug_api,
            debug_prn = debug_prn,
            logger= logger
        )
        for instance in config_ls
    ], n = 10)

    return pd.DataFrame(config_ls)

#### sample implementation of `get_domains_with_instance_auth`


In [10]:
import os

config_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

default_auth = dmda.DomoTokenAuth(
    domo_instance="default", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)


class AuthEnum(Enum):
    """enum_name must match values in auth_match_col from config_sql query"""
    test_1 = dmda.DomoFullAuth(
        domo_instance="domo-community", 
        domo_password =os.environ["DOJO_PASSWORD"],
        domo_username = 'jae@onyxreporting.com') 
    test_0 = dmda.DomoTokenAuth(
        domo_instance="domo-community-test", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"])
    config_1 = dmda.DomoTokenAuth(
        domo_instance="domo-community-config", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"])    
    config_0 = dmda.DomoTokenAuth(
        domo_instance="domo-community-config-test", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"])


logger = lc.Logger(app_name="test_retrieve_company")

res = await GetInstanceConfig.get_domains_with_instance_auth(
    config_auth=config_auth,
    config_dataset_id="8d2a8055-7918-4039-b67d-361647e96ea8",
    config_sql="""
    SELECT domain as domo_instance, 'test_1'  as auth_match_col, 1 as config_exception_pw from Table
    """,
    debug_prn=True,
    debug_log=False,
    debug_api=True,
    logger=logger,
    default_auth=default_auth,
    auth_enum=AuthEnum,
)

pd.DataFrame(res)
# pd.DataFrame(logger.logs)

🐛 debugging get_data


{'body': None,
 'function_name': 'get_from_id',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '83ece44f1451d4b581e1191f98cd411164f0b5b6ad2755b3'},
 'method': 'GET',
 'params': None,
 'parent_class': 'GetInstanceConfig',
 'url': 'https://domo-community.domo.com/api/data/v3/datasources/8d2a8055-7918-4039-b67d-361647e96ea8'}
get_data: no body
get_data_response <Response [200 OK]>
⚙️ START - Retrieving company list 
https://domo-community.domo.com/datasources/8d2a8055-7918-4039-b67d-361647e96ea8/details/overview using 

    SELECT domain as domo_instance, 'test_1'  as auth_match_col, 1 as config_exception_pw from Table
    
🐛 debugging get_data
{'body': {'sql': '\n'
                 "    SELECT domain as domo_instance, 'test_1'  as "
                 'auth_match_col, 1 as config_exception_pw from Table\n'
                 '     limit 100 offset 0'},
 'fu

,domo_instance,auth_match_col,config_exception_pw,is_valid,is_v2
0,domo-dojo,test_1,1,1,0


In [11]:
# | export
class InvalidAccountNameError(Exception):
    """raised when account name does not follow format string"""

    def __init__(self, account_name=None, regex_pattern=None):
        account_str = f'"{account_name}" '
        regex_str = f'"{regex_pattern}"'

        message = f"string {account_str if account_name else ''}does not match regex pattern {regex_str or ''}"
        self.message = message

        super().__init__(self.message)


@patch_to(DomoJupyterAccount_InstanceAuth, cls_method=True)
def get_domo_instance_auth_account(
    cls: DomoJupyterAccount_InstanceAuth,
    account_name: str,  # domojupyter account to retrieve
    # Domo's domojupyter module, pass in b/c can only be retrieved inside Domo jupyter notebook environment
    domojupyter_fn: callable,
    # set the domo_instance or retrieve from the domojupyter_account credential store
    domo_instance=None,
):
    """
    retrieves Abstract Credential Store from DomoJupyter environment.
    expects credentials property to contain DOMO_USERNAME, DOMO_PASSWORD, or DOMO_ACCESS_TOKEN, and (optional) DOMO_INSTANCE
    """

    if not cls._test_regex_mask(account_name, cls.account_name_mask):
        raise InvalidAccountNameError(
            account_name=account_name, regex_pattern=cls.account_name_mask
        )

    account_properties, dj_account = get_jupyter_account(
        account_name, domojupyter_fn=domojupyter_fn
    )

    if account_properties != ["credentials"]:
        raise InvalidAccountTypeError(
            account_name=account_name, account_type="abstract_credential_store"
        )

    creds = json.loads(dj_account.get("credentials"))

    return cls(
        account_name=account_name,
        raw_cred=creds,
        domo_username=creds.get("DOMO_USERNAME"),
        domo_password=creds.get("DOMO_PASSWORD"),
        domo_access_token=creds.get("DOMO_ACCESS_TOKEN"),
        domo_instance=domo_instance or creds.get("DOMO_INSTANCE"),
    )

In [12]:
# show_doc(DomoJupyterAccount_InstanceAuth.get_domo_account)

In [13]:
# | export
class GenerateAuth_InvalidDomoInstanceList(Exception):
    def __init__(self):
        message = "provide a list of domo_instances"
        super().__init__(message)


class GenerateAuth_CredentialsNotProvided(Exception):
    def __init__(self):
        message = "object does not have a valid combination of credentials (access_token or username and password)"
        super().__init__(message)


@patch_to(DomoJupyterAccount_InstanceAuth)
def generate_auth(self, domo_instance):
    if self.domo_access_token:
        auth = dmda.DomoTokenAuth(
            domo_instance=domo_instance, domo_access_token=self.domo_access_token
        )

    elif self.domo_username and self.domo_password:

        auth = dmda.DomoFullAuth(
            domo_instance=domo_instance,
            domo_username=self.domo_username,
            domo_password=self.domo_password,
        )

    else:
        raise GenerateAuth_CredentialsNotProvided()

    return auth


@patch_to(DomoJupyterAccount_InstanceAuth)
def generate_auth_ls(
    self: DomoJupyterAccount_InstanceAuth,
    domo_instance_ls: list[str] = None,  # list of domo_instances
) -> list[dmda.DomoAuth]:  # list of domo auth objects

    """for every domo_instance in domo_instance_ls generates an DomoAuth object"""

    # reset internal lists
    self.domo_instance = None

    self.domo_instance_ls = list(set(domo_instance_ls or self.domo_instance_ls))

    if not self.domo_instance_ls:
        raise GenerateAuth_InvalidDomoInstanceList()

    self.auth_ls = []
    for domo_instance in self.domo_instance_ls:
        auth = self.generate_auth(domo_instance)

        self.auth_ls.append(auth)

    return self.auth_ls

In [14]:
# show_doc(DomoJupyterAccount_InstanceAuth.generate_auth_ls)

In [15]:
DomoJupyterAccount_InstanceAuth(account_name="test")

DomoJupyterAccount_InstanceAuth(account_name='test', domo_username=None)

In [16]:
# | hide
import nbdev

nbdev.nbdev_export()